# Panda 2

## Partie A — Débutants
1. Charger `customers_dirty.csv` et afficher le nombre de lignes et colonnes.

In [83]:
import pandas as pd

df = pd.read_csv('./data/customers_dirty.csv')
dfc = pd.read_csv('./data/customers_dirty copy.csv')
dfc.shape
df.shape

(5, 4)

In [85]:
df

,id,name,age,city
0,1,Alice,25.0,paris
1,2,Bob,NaN,LYON
2,3,Charlie,40.0,Marseille
3,4,Alice,25.0,paris
4,5,Ethan,32.0,NaN


2. Vérifier combien de valeurs manquantes par colonne.  

In [86]:
df.isna().sum()
dfc.isna().sum()

id      0
name    0
age     1
city    1
dtype: int64

3. Supprimer toutes les lignes avec des valeurs manquantes.  

In [87]:
dfc_clean = dfc.dropna(how='any')
dfc_clean

,id,name,age,city
0,1,Alice,25.0,paris
2,3,Charlie,40.0,Marseille
3,4,Alice,25.0,paris


4. Remplacer les valeurs manquantes d’âge par la moyenne des âges.  

In [88]:
mean_age = df['age'].mean()

df['age'] = df['age'].fillna(mean_age)
dfc['age'] = df['age'].fillna(mean_age)
df

,id,name,age,city
0,1,Alice,25.0,paris
1,2,Bob,30.5,LYON
2,3,Charlie,40.0,Marseille
3,4,Alice,25.0,paris
4,5,Ethan,32.0,NaN


5. Uniformiser la colonne `city` (première lettre en majuscule).

In [89]:
df['city'] = df['city'].str.capitalize()
dfc['city'] = dfc['city'].str.capitalize()
df

,id,name,age,city
0,1,Alice,25.0,Paris
1,2,Bob,30.5,Lyon
2,3,Charlie,40.0,Marseille
3,4,Alice,25.0,Paris
4,5,Ethan,32.0,NaN


6. Supprimer les doublons.

In [90]:
dfc = dfc.drop_duplicates(subset=['name', 'age', 'city'])
df = df.drop_duplicates(subset=['name', 'age', 'city'])
# dfd
df

,id,name,age,city
0,1,Alice,25.0,Paris
1,2,Bob,30.5,Lyon
2,3,Charlie,40.0,Marseille
4,5,Ethan,32.0,NaN


*(Optionnel)* Compter le nombre de clients uniques par ville.


## Partie B — Challenge
7. Créer une nouvelle colonne `is_adult` (âge >= 18).  

In [91]:
# df['is_adult'] = df['age'] > 18
# df
dfc.loc[:, "is_adult"] = dfc["age"] > 18
df.loc[:, "is_adult"] = df["age"] > 18
df

/tmp/ipykernel_5845/2480313306.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfc.loc[:, "is_adult"] = dfc["age"] > 18
/tmp/ipykernel_5845/2480313306.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, "is_adult"] = df["age"] > 18


,id,name,age,city,is_adult
0,1,Alice,25.0,Paris,True
1,2,Bob,30.5,Lyon,True
2,3,Charlie,40.0,Marseille,True
4,5,Ethan,32.0,NaN,True


8. Récupérer depuis `marketing.db` le nombre de commandes par client.

In [92]:
import sqlite3
import pandas as pd
# Create your connection.
cnx = sqlite3.connect('./data/marketing.db')

# db_orders = pd.read_sql_query("SELECT * FROM orders", cnx)
# db_orders
orders = pd.read_sql_query("SELECT * FROM orders", cnx)
customers = pd.read_sql_query("SELECT * FROM customers", cnx)
merged = pd.merge(orders, customers, right_on='id', left_on='customer_id')
# merged
order_counts = merged.groupby('name')['customer_id'].count().reset_index()
order_counts

,name,customer_id
0,Alice,2
1,Bob,1
2,Charlie,1


9. Fusionner ce résultat avec le CSV nettoyé. 

In [ ]:
# Compter les commandes par client_id
order_counts = orders.groupby('customer_id').size().reset_index(name='order_count')
# order_counts
# dfc
# Fusionner avec ton CSV (sur id = customer_id)
merged = pd.merge(dfc, order_counts, how='left', left_on='id', right_on='customer_id')
# merged
# Supprimer la colonne 'customer_id' (inutile)
merged = merged.drop(columns=['customer_id'])
# merged
# Remplacer NaN par 0 (clients sans commandes)
merged['order_count'] = merged['order_count'].fillna(0).astype(int)
# merged
# Sauvegarder dans un nouveau CSV
merged.to_csv("./data/customers_dirty copy.csv", index=False)

print(merged)


# dfc

   id     name   age       city  is_adult  order_count
0   1    Alice  25.0      Paris      True            2
1   2      Bob  30.5       Lyon      True            1
2   3  Charlie  40.0  Marseille      True            1
3   5    Ethan  32.0        NaN      True            0


10. Créer une colonne `fidelite` : "fidèle" si nb_commandes > 1, sinon "nouveau".

11. Exporter le résultat final dans `data/customers_clean.csv`. 

*(Optionnel)* Lister les 3 clients les plus fidèles.  